In [ ]:
from pyquaternion import Quaternion
import numpy as np
from mono_depth_estimator import create_depth_frames
import math

In [ ]:
# Taken from https://github.com/apperception-db/apperception/blob/main/apperception/lens.py#L32
def create_transform(resolution, cam_origin, yaw):
	x, y = resolution
	cam_origin = cam_origin
	cam_x, cam_y, cam_z = cam_origin

	yaw, pitch, roll = np.deg2rad(yaw), 0, 0 # assume perfect camera for no. TODO: Fix this
	# yaw, pitch, roll = np.array(Quaternion(cam_rotation).yaw_pitch_roll)
	# yaw, pitch, roll = np.deg2rad(yaw), np.deg2rad(pitch), np.deg2rad(roll)
	R_1, R_2, R_3 = np.cos(pitch)*np.cos(yaw), np.cos(pitch)*np.sin(yaw), -np.sin(pitch)
	R_4 = np.sin(roll)*np.sin(pitch)*np.cos(yaw) - np.cos(roll)*np.sin(yaw)
	R_5 = np.sin(roll)*np.sin(pitch)*np.sin(yaw) + np.cos(roll)*np.cos(yaw)
	R_6 = np.sin(roll)*np.cos(pitch)

	R_7 = np.cos(roll)*np.sin(pitch)*np.cos(yaw) + np.sin(roll)*np.sin(yaw)
	R_8 = np.cos(roll)*np.sin(pitch)*np.sin(yaw) - np.sin(roll)*np.cos(yaw)
	R_9 = np.cos(roll)*np.cos(pitch)

	rotation_mat = np.matrix([[R_1, R_2, R_3],
		[R_4, R_5, R_6], 
		[R_7, R_8, R_9]])
	cam_org_vec = np.matrix([[cam_x], [cam_y], [cam_z]])
	col_vec = np.ravel(rotation_mat @ cam_org_vec)
	col_x, col_y, col_z = col_vec
	transform = np.matrix([[R_1, R_2, R_3, -col_x],
		[R_4, R_5, R_6, -col_y], 
		[R_7, R_8, R_9, -col_z],
		[0, 0, 0, 1]
		])
	return transform

In [ ]:
def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw)

rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
def get_camera_heading(rotation):
    return -get_heading(rot.rotate(rotation)) + math.pi / 2

In [ ]:
def transform_to_world(frame_coordinates, cam_trans_abs, camera_heading, depths):
    resolution = (1600, 900)
    transform = create_transform(resolution, cam_trans_abs, camera_heading)
    
    depth = depths[frame_coordinates]
    x, y = frame_coordinates
    pixel = np.matrix([[x], [y], [depth], [0]])
    return transform @ pixel

In [ ]:
image = np.random.rand(1, 1600, 900, 3)

In [ ]:
depths = create_depth_frames(image)[0]

In [ ]:
cam_trans_abs = (400, 100, 0.5)
cam_rotation = Quaternion([0.50772417,-0.49733922,0.49837166,-0.4964832]) # heading of 96 degrees
camera_heading = 0 # get_camera_heading(cam_rotation)
frame_coordinates = (800, 450)

In [ ]:
depths[frame_coordinates]

In [ ]:
np.set_printoptions(suppress=True)
transform_to_world(frame_coordinates, cam_trans_abs, camera_heading, depths)